In [2]:
import queue

class Symbol:
    '''Represents a logical symbol and its truth value.'''
    
    def __init__(self, symbol, value = True):
        self.symbol = symbol
        self.value = value
    
    def __repr__(self):
        return(self.symbol if self.value else "-" + self.symbol)
    
    def __hash__(self):
        return(hash(self.symbol))
    
    def __eq__(self, other):
        return(self.symbol == other.symbol)
    
    def __neg__(self):
        return(Symbol(self.symbol, value = not self.value))

class DefiniteClause:
    '''Represents a definite clause.
    
        Inputs: 
            literals (iterable of Symbols): the components of the clause. 
            All symbols but one should be negated.'''

    def __init__(self, literals):
        self.premise = [lit for lit in literals if not lit.value]
        self.head = next(lit for lit in literals if lit.value)
        self.symbols = [lit if lit.value else -lit for lit in literals]
        
    def __hash__(self): 
        return(hash(tuple(lit for lit in self.premise)))
    
    def __len__(self):
        return(len(self.symbols))
        
    def __repr__(self):
        if len(self) > 1:
            return(r' ^ '.join(str(lit)[1:] for lit in self.premise) + ' -> ' + str(self.head))
        else:
            return(str(self.head))
        
def forward_chaining(KB, q):
    '''The forward chaining algorithm to assert entailment.
    
        Inputs:
            KB (iterable of DefinitiveClauses): The compoments of the KB.
            q (Symbol): The symbol whose truth we're interested in asserting.''' 
    
    # The number of symbols in each clause’s premise
    count = {cl : len(cl.premise) for cl in KB}
    
    # Initially false for all symbols
    inferred = {sb : False for cl in KB for sb in cl.symbols}
    
    # a queue of symbols, initially symbols known to be true in KB
    agenda = queue.Queue()
    for cl in KB:
        if len(cl) == 1:
            agenda.put(cl.head)
    
    while not agenda.empty():  # while not empty
        
        # Get the next symbol inthe agenda
        p = agenda.get()
        
        # If it's the query, we proof is successful
        if p == q: return(True)
        
        # If the symbol has not been inferred before
        if not inferred[p]:
            inferred[p] = True
            
            # For each clause in the KB which contains the symbol in its premise
            for cl in KB:
                if p in cl.premise:
                    
                    # Decrease the count of remaining unknown literals in the premimse
                    count[cl] -= 1
                    
                    # If all literals in the premise have been found to be true
                    if count[cl] == 0:
                        
                        # Add the head of the clause to the agenda
                        agenda.put(cl.head)
    return(False)

In [8]:
A = Symbol("A")
B = Symbol("B")
C = Symbol("C")
R1 = DefiniteClause([-A, -B, C])
R2 = DefiniteClause([A])
R3 = DefiniteClause([B])
forward_chaining([R1, R2, R3], C)

True